In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [6]:
config = get_config(nb_name)

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [8]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [9]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [10]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [11]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [12]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [13]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:  \
            Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   
499     8.505648    0  498  113.06  572  112.63  0.42  0.02  105.59   
5489    4.483361   10  498  111.40  714  110.38  1.02  0.00  103.89   
10479   5.222531   20  498  111.00  798  109.68  1.30  0.01  103.49   
15469   5.469910   30  498  110.75  837  109.25  1.49  0.01  103.31   
20459   5.052303   40  498  110.58  852  108.95  1.62  0.01  103.19   
25449   5.622214   50  498  110.46  857  108.73  1.72  0.01  103.08   
30439   5.288658   60  498  110.36  857  108.56  1.80  0.01  103.04   
35429   5.443166   70  498  110.28  854  108.42  1.86  0.00  102.93   
40419   5.012738   80  498  110.21  850  108.29  1.91  0.00  102.95   
45409   5.700519   90  498  110.15  844  108.19  1.95  0.00  102.91   
50399   5.067049  100  498  110.10  838  108.11  1.99  0.00  102.86   
55389   5.429038  110  498  110.06  833  108.04  2.02  0.00  102.84   
60379   5.550978  120  498  110.03  827  107.97  2.05  0.00  102.84   
65369   5.388807  130  498  110.00  822  107.92  2.07  0.00  102.82   
70359   5.036436  140  498  109.97  817  107.87  2.10  0.00  102.81   
75349   5.468581  150  498  109.94  812  107.82  2.12  0.00  102.81   
80339   5.085728  160  498  109.92  808  107.78  2.13  0.00  102.83   
85329   5.240547  170  498  109.90  804  107.75  2.15  0.00  102.82   
90319   4.935555  180  498  109.88  800  107.71  2.16  0.00  102.81   
95309   5.608860  190  498  109.86  796  107.68  2.18  0.00  102.79   
100299  5.088680  200  498  109.85  793  107.65  2.19  0.00  102.79   
105289  5.480487  210  498  109.83  790  107.63  2.20  0.00  102.81   
110279  5.198197  220  498  109.82  787  107.60  2.21  0.00  102.80   
115269  5.496094  230  498  109.80  784  107.58  2.22  0.00  102.76   
120259  5.110715  240  498  109.79  781  107.56  2.23  0.00  102.77   
125249  5.931230  250  498  109.78  778  107.54  2.24  0.00  102.76   
130239  5.520617  260  498  109.77  776  107.52  2.25  0.00  102.75   
135229  5.934197  270  498  109.76  773  107.50  2.26  0.00  102.77   
140219  5.121685  280  498  109.75  771  107.48  2.26  0.00  102.82   
145209  5.922662  290  498  109.74  769  107.47  2.27  0.00  102.73   
150199  5.104421  300  498  109.73  767  107.45  2.28  0.00  102.80   
155189  8.536882  310  498  109.72  765  107.44  2.28  0.00  102.75   
160179  6.854395  320  498  109.71  763  107.42  2.29  0.00  102.79   
165169  7.906287  330  498  109.71  761  107.41  2.30  0.00  102.74   
170159  5.730911  340  498  109.70  759  107.40  2.30  0.00  102.77   
175149  6.414472  350  498  109.69  757  107.39  2.31  0.00  102.77   
180139  5.366204  360  498  109.69  756  107.38  2.31  0.00  102.76   
185129  6.324257  370  498  109.68  754  107.36  2.32  0.00  102.74   
190119  7.094095  380  498  109.68  752  107.36  2.32  0.00  102.77   

                                         TEST:             SPEC:              \
               PPL     NLL    KL   REG    LOSS         PPL     1     2     3   
499     537.874756  105.25  0.34  0.00  104.00  542.625610  0.19  0.25  0.31   
5489    491.848694  102.57  1.32  0.00  102.43  494.382996  0.26  0.47  0.51   
10479   482.622589  101.87  1.62  0.00  102.02  484.953461  0.25  0.59  0.53   
15469   478.295837  101.51  1.81  0.00  101.86  481.406281  0.26  0.61  0.56   
20459   474.634796  101.27  1.92  0.00  101.75  478.031799  0.27  0.62  0.55   
25449   471.840271  101.07  2.01  0.00  101.66  476.103210  0.28  0.62  0.56   
30439   470.308563  100.97  2.07  0.00  101.63  475.125061  0.28  0.63  0.56   
35429   467.321960  100.80  2.13  0.00  101.50  471.674713  0.29  0.63  0.56   
40419   468.625366  100.79  2.16  0.00  101.50  471.674713  0.30  0.63  0.54   
45409   466.897858  100.72  2.18  0.00  101.49  471.788086  0.28  0.64  0.57   
50399   465.590607  100.65  2.21  0.00  101.41  469.278992  0.29  0.63  0.55   
55389   465.487396  100.61  2.23  0.00  101.37  468.996460  0.30  0.63  0.56

0 R: 1.000 P: 0.346 carry pockets back nice strap work lot quality - small
   1 R: 0.232 P: 0.116 pocket ipad room mouse charger power perfect small perfectly netbook
     13 R: 0.036 P: 0.036 ; & pro size air big perfectly price bought ...
     12 R: 0.080 P: 0.080 ! love perfect price recommend bought color ... buy perfectly
   3 R: 0.225 P: 0.102 sleeve protection inside air nice protect snug inch zipper material
     31 R: 0.065 P: 0.065 color cover ! love perfectly mac pro picture blue pink
     33 R: 0.057 P: 0.057 smell retina $ air pro reviews sleeve price inch item
   4 R: 0.076 P: 0.029 months broke zipper started weeks year month handle week coming
     41 R: 0.046 P: 0.046 ! return ... $ received item money quality amazon buy
   2 R: 0.122 P: 0.051 bottom top plastic corners piece part open place back nice
     21 R: 0.071 P: 0.071 cover keyboard color screen ! hard easy love protector bottom
{0: [1, 3, 2, 5], 1: [12, 11], 3: [31, 33, 32], 2: [21, 22], 5: [51]}
